In [ ]:
# Data Collection - Stock Prices & News Headlines

This notebook covers:
1. Downloading historical stock price data from Yahoo Finance
2. Collecting news headlines for sentiment analysis
3. Data preprocessing and storage


In [ ]:
## 1. Setup and Imports


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Create data directory if it doesn't exist
Path('../data').mkdir(exist_ok=True)

print("Setup complete!")


In [ ]:
## 2. Download Stock Price Data


In [ ]:
# Define stock tickers - starting with major tech stocks
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA', 'NFLX', 'CRM', 'ADBE']

# Date range for historical data
start_date = "2022-01-01"
end_date = "2024-12-31"

print(f"Downloading price data for {len(tickers)} stocks from {start_date} to {end_date}...")
